<a href="https://colab.research.google.com/github/ds-personalization/final-project-qrdecomposition_final/blob/dev/eggs/model_without_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir('/content/drive/MyDrive/personalization/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pytorch_widedeep

     |████████████████████████████████| 92kB 5.1MB/s 


In [180]:
import pandas as pd
import numpy as np
import pytorch_widedeep as wd
from pytorch_widedeep.preprocessing import WidePreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep

In [181]:
train_path = '../data/interim/train_0.75_0.25.csv'
test_path = '../data/interim/test_0.75_0.25.csv'
genre_path = '../data/raw/movies.csv'
tag_path = '../data/raw/tags.csv'

In [182]:
train, test, genre, tags = pd.read_csv(train_path), pd.read_csv(test_path), pd.read_csv(genre_path), pd.read_csv(tag_path)
train = train.merge(genre).merge(tags.drop('timestamp', axis=1))
test = test.merge(genre).merge(tags.drop('timestamp', axis=1))

In [183]:
from pytorch_widedeep.preprocessing import WidePreprocessor, TextPreprocessor, DensePreprocessor

In [184]:
wide_cols = ['genres', 'tag']
crossed_cols = [('genres', 'tag')]
embs = [('userId', 32), ('movieId', 32)]
target_cols = 'rating'
train_target = train[target_cols].values

In [185]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
train_wide = preprocess_wide.fit_transform(train)
test_wide = preprocess_wide.transform(test)

In [186]:
# DEEP
preprocess_deep = DensePreprocessor(embed_cols=embs)
train_id = preprocess_deep.fit_transform(train)
test_id = preprocess_deep.transform(test)

In [187]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [188]:
cv = CountVectorizer()
tfidfv = TfidfVectorizer()

In [189]:
svd = TruncatedSVD(n_components=5)
train_genre = svd.fit_transform(cv.fit_transform(train['genres']))
test_genre = svd.fit_transform(cv.transform(test['genres']))

In [190]:
# svd = TruncatedSVD(n_components=5)
# train_tag = svd.fit_transform(tfidfv.fit_transform(train['tag']))
# test_tag = svd.fit_transform(tfidfv.transform(test['tag']))

In [191]:
train_deep = np.hstack((train_id, train_genre))
test_deep = np.hstack((test_id, test_genre))

In [192]:
wide = Wide(wide_dim=np.unique(train_wide).shape[0], pred_dim=1)

deepdense = DeepDense(hidden_layers=[128,64, 32], 
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input)

model = WideDeep(wide=wide, deepdense=deepdense)

In [193]:
model.compile(method='regression')

In [194]:
model.fit(X_wide=train_wide, X_deep=train_deep, target=train_target, n_epochs=30, batch_size=256, val_split=0.1)

  0%|          | 0/32 [00:00<?, ?it/s]

Training


valid: 100%|██████████| 4/4 [00:00<00:00, 35.95it/s, loss=0.707]


In [195]:
pred = model.predict(X_wide=test_wide, X_deep=test_deep)

predict: 100%|██████████| 17/17 [00:00<00:00, 96.16it/s]


In [196]:
from sklearn.metrics import mean_squared_error

In [197]:
np.sqrt(mean_squared_error(pred, test.rating))

1.2995823489100773